In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
#from census import Census
import gmaps
from scipy.stats import linregress
import numpy as np
import os
from apikeys import g_key


In [2]:
# Load the combined CSV file
combined_df = pd.read_csv("combined.csv")

# Display sample data
combined_df

,Unnamed: 0,State,Name,Lat,Lng,Population,Median Age,Household Income,Per Capita Income,Poverty Count,...,owned_households_%,rented_households_%,homes_#_powered_by_solar,homes_%_powered_by_solar,state_elect_from_solar_%,solar_companies_in_state,MW_currently_installed,MW_growth_projection_5_years,MW_total_in_5_years,MW_output_%_in_5_years
0,15,56,Wyoming,41.1403,-104.8202,581348,38.0,65304,34415,61006,...,71.4,28.6,22766,10.461883,0.0046,8,143,234,377,263.636364
1,25,30,Montana,45.4361,-104.0655,1061705,40.1,56539,32463,132476,...,69.5,30.5,16986,4.726473,0.0030,28,127,1449,1576,1240.944882
2,9,38,North Dakota,46.8210,-100.7816,760394,35.2,65315,36289,77491,...,63.1,36.9,147,0.058257,0.0001,8,2,298,300,15000.000000
3,50,46,South Dakota,44.3671,-100.3464,879336,37.2,59896,31415,108863,...,69.4,30.6,232,0.081563,0.0002,11,2,252,254,12700.000000
4,45,35,New Mexico,35.6824,-105.9399,2097021,38.1,51243,27945,381026,...,69.5,30.5,344556,45.448622,0.0605,69,1452,4365,5817,400.619835


In [3]:
# Configure gmaps
gmaps.configure(apikey=g_key)

# Store the latitude and longitude in locations and store the humidity to be used as the weight
locations = combined_df[["Lat", "Lng"]].astype(float)
futureIncrease = combined_df["MW_growth_projection_5_years"].astype(float)
radiation = combined_df["daily_solar_radiation"].astype(float)

In [4]:
# Plot Heatmap 
fig = gmaps.figure()

# Create heat layer
future_layer = gmaps.heatmap_layer(locations, weights = futureIncrease, dissipating = False, 
                                 max_intensity = max(futureIncrease), point_radius = 4)

# Add heat layer 
fig.add_layer(future_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Narrow down cities for ideal weather conditions and drop any null values
ideal_solar = combined_df.loc[(combined_df["daily_solar_radiation"] >= 4) & (combined_df["electricity_price_residential"] >= 12) & (combined_df["pop_density"] <= 80)].dropna()

# Reset the index
ideal_solar = ideal_solar.reset_index(drop = True)

# Display results
ideal_solar

,Unnamed: 0,State,Name,Lat,Lng,Population,Median Age,Household Income,Per Capita Income,Poverty Count,...,owned_households_%,rented_households_%,homes_#_powered_by_solar,homes_%_powered_by_solar,state_elect_from_solar_%,solar_companies_in_state,MW_currently_installed,MW_growth_projection_5_years,MW_total_in_5_years,MW_output_%_in_5_years
0,45,35,New Mexico,35.6824,-105.9399,2097021,38.1,51243,27945,381026,...,69.5,30.5,344556,45.448622,0.0605,69,1452,4365,5817,400.619835
1,46,32,Nevada,39.1641,-119.7661,3030281,38.2,62043,32629,381695,...,59.1,40.9,857303,113.331209,0.2187,87,5003,7707,12710,254.047572
2,36,20,Kansas,39.0483,-95.6781,2912619,36.9,61091,32798,323644,...,67.0,33.0,15102,1.710961,0.0022,42,106,1308,1414,1333.962264
3,28,8,Colorado,39.7393,-104.9848,5684926,36.9,75231,39545,544232,...,66.8,33.2,440055,21.927047,0.0595,330,2268,3946,6214,273.985891
4,16,4,Arizona,33.4481,-112.0970,7174064,37.9,61529,32340,990528,...,67.6,32.4,945767,39.447639,0.0970,296,5984,5342,11326,189.271390


In [6]:
# Plot Heatmap 
fig = gmaps.figure()

# Create heat layer
ideal_solar = gmaps.heatmap_layer(locations, weights = radiation, dissipating = False, 
                                 max_intensity = max(radiation), point_radius = 4)

# Add heat layer 
fig.add_layer(ideal_solar)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))